In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import os

# --- 1. Configuração do Spark e Variáveis ---
spark = SparkSession.builder \
    .appName("GoldLayerModelingOptimized") \
    .getOrCreate()

SILVER_PATH = 'hdfs://namenode:9870/datalake/silver/who_data_silver.parquet'
GOLD_PATH = 'hdfs://namenode:9870/datalake/gold/correlation_ready_data.parquet'

def load_silver_data(path):
    """Carrega os dados limpos da camada Silver do HDFS."""
    print(f"Lendo dados limpos de: {path}")
    try:
        df_silver = spark.read.parquet(path)
        return df_silver
    except Exception as e:
        print(f"ERRO ao ler o Parquet em {path}: {e}")
        return spark.createDataFrame([], schema=None)

def transform_to_gold(df_silver):
    """Pivotar os dados para o modelo final (wide format)."""
    if df_silver.count() == 0:
        return spark.createDataFrame([], schema=None)

    print("Iniciando modelagem Gold Otimizada (Pivotagem)...")

    # Pivotagem direta. Usamos 'first' como função de agregação,
    # pois não deve haver duplicatas no Silver (após o filtro por BOTHSEXES)
    df_gold = df_silver.groupBy("Country_Code", "Year").pivot("Indicator_Name").agg(col("Value").alias("first"))

    # Os nomes das colunas são os nomes dos indicadores
    df_gold = df_gold.filter(
        col("Taxa_Mortalidade_Suicidio").isNotNull() & 
        col("Prevalencia_Transtorno_Humor").isNotNull()
    )
    
    print(f"Modelagem Gold concluída. Total de linhas prontas para correlação: {df_gold.count()}")
    return df_gold

def save_gold_data(df_gold, file_path):
    """Salva o DataFrame final no formato Parquet no HDFS."""
    if df_gold.count() > 0:
        print(f"Salvando dados agregados (Camada Gold) em: {file_path}")
        df_gold.write.mode("overwrite").parquet(file_path)
        print("Dados salvos com sucesso!")


if __name__ == "__main__":
    df_silver = load_silver_data(SILVER_PATH)
    if df_silver.count() > 0:
        df_gold = transform_to_gold(df_silver)
        save_gold_data(df_gold, GOLD_PATH)
    else:
        print("O processamento Gold não pode ser executado, pois a camada Silver está vazia.")
    
    spark.stop()